In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Program Files\\spark\\spark-3.4.1-bin-hadoop3'

In [8]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Broadcast_variable').master('local[2]').getOrCreate()

In [9]:
states = {"NY":"New York", "CA":"California", "FL":"Florida"}

In [39]:
broadcastStates = spark.sparkContext.broadcast(states)
broadcastStates.value.keys()

dict_keys(['NY', 'CA', 'FL'])

In [18]:
data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL")
  ]
columns = ["firstname","lastname","country","state"]
df = spark.createDataFrame(data = data, schema = columns)
df.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)



In [19]:
df.show(truncate=False)

+---------+--------+-------+-----+
|firstname|lastname|country|state|
+---------+--------+-------+-----+
|James    |Smith   |USA    |CA   |
|Michael  |Rose    |USA    |NY   |
|Robert   |Williams|USA    |CA   |
|Maria    |Jones   |USA    |FL   |
+---------+--------+-------+-----+



In [20]:
def state_convert(code):
    return broadcastStates.value[code]

In [29]:
result = df.rdd.map(lambda x: (*x[0:3],state_convert(x[3]))).toDF(columns)
result.show(truncate=False)

+---------+--------+-------+----------+
|firstname|lastname|country|state     |
+---------+--------+-------+----------+
|James    |Smith   |USA    |California|
|Michael  |Rose    |USA    |New York  |
|Robert   |Williams|USA    |California|
|Maria    |Jones   |USA    |Florida   |
+---------+--------+-------+----------+



In [49]:
# Broadcast variable on filter
filteDf= df.where((df['state'].isin(*broadcastStates.value.keys())))

In [50]:
filteDf.show()

+---------+--------+-------+-----+
|firstname|lastname|country|state|
+---------+--------+-------+-----+
|    James|   Smith|    USA|   CA|
|  Michael|    Rose|    USA|   NY|
|   Robert|Williams|    USA|   CA|
|    Maria|   Jones|    USA|   FL|
+---------+--------+-------+-----+

